In [0]:

#raw data

df1=spark.read.parquet("s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2019-10-23/platform=1/").filter("search_volume>=100 and country_code='US'")#.filter("keyword_id=5198892 and country_code='NL' and platform=1 and device_id=0 ")#.show(20,False)
df1.show()
# df2=spark.read.parquet("s3://prod.appannie.mktint.data/oss/KEYWORD_DIFFICULTY/routine/granularity=daily/date=2018-07-07/platform=1/").filter("keyword_id='4999617' and country_code='US'")#.filter("keyword_id=5198892 and country_code='NL' and platform=1 and device_id=0 ")#.show(20,False)
# df2.show()
# result=df1.join(df2, df1.keyword_id==df2.keyword_id, 'outer').show()


# df2=spark.read.parquet("s3://prod.appannie.mktint.data/oss/KEYWORD_DIFFICULTY/routine/granularity=daily/date=2018-07-07/platform=1/").filter("keyword_id='4999617' and country_code='US'")#.filter("keyword_id=5198892 and country_code='NL' and platform=1 and device_id=0 ")#.show(20,False)

# print result

# spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.keyword-difficulty-search-volume.v1/fact/granularity=daily/date=2017-07-06").filter("device_code='android-all' and dev").show(20,False)



# df2=spark.read.parquet("s3://prod.appannie.mktint.data/oss/KEYWORD_DIFFICULTY/routine/granularity=daily/date=2016-10-11/").filter("country_code='US' and platform=2 and device_id=1 and difficulty is null ").count()
# print df2


# df1=spark.read.parquet("s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2017-07-06/platform=1/").filter("country_code='us'").show()#.filter("keyword_id=5198892 and country_code='NL' and platform=1 and device_id=0 ")#.show(20,False)



In [0]:

import unittest
from random import choice, randint

from aaintdatapipeline.application.app_qa.common.utils import get_month_start_end_date
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import IOS_PHONE, IOS_TABLET, \
    ANDROID_ALL, ASO_TS_RAW_DATA_HEAD
from aaintdatapipeline.application.app_qa.data_validation_v1.pysparktest import PySparkTest
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_random_date, get_aso_ts_raw_df, get_aso_svi_raw_df, get_aso_kd_raw_df,
    check_parquet_exist, get_date_list_from_granularity, get_aso_ts_unified_data_path)
from aaintdatapipeline.core.fs.device import S3Bucket
from aaintdatapipeline.core.conf import Conf
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_aso_citus_connection, datetime_to_string)
from aaintdatapipeline.application.app_qa.data_validation_v1.pysparktest import PySparkTest

from aaintdatapipeline.application.app_qa.data_validation_v1.constants import \
    urn
from decimal import Decimal
import decimal

mkt_new_bucket = "b2c-prod-int-data-pipeline-unified-paid-aso"
ts_unified_path = \
    "s3://{}/unified/app-tech.market.aso-keyword-trafficshare.v1/fact/granularity=daily/date={}/device_code={}/"
svi_kd_unified_path = "s3://{}/unified/app-tech.market.aso-keyword-difficulty-search-volume.v1/fact/" \
                      "granularity=monthly/date={}/device_code={}/"


date = "2018-06-01"
platform = "1"
device_code="ios-tablet"

aso_sv_kd_raw_bucket = "prod.appannie.mktint.data"
aso_sv_raw_bucket_path = "s3://{}/oss/SEARCH_VOLUME/routine/granularity=daily/date={}/platform={}/"
aso_kd_raw_bucket_path = "s3://{}/oss/KEYWORD_DIFFICULTY/routine/granularity=daily/date={}/platform={}"


aso_sv_kd_unified_bucket = "b2c-prod-data-pipeline-unified-aso"
aso_sv_kd_unified_bucket_path = "s3://{}/unified/app-tech.aso.keyword-difficulty-search-volume.v1/fact/granularity=daily/date={}/device_code={}/"




def get_aso_sv_raw_df_root(path, aso_sv_raw_bucket, date, platform):
    return path.format(aso_sv_raw_bucket, date, platform)

def get_aso_kd_raw_df_root(path, aso_kd_raw_bucket, date, platform):
    return path.format(aso_kd_raw_bucket, date, platform)

def get_aso_sv_kd_unified_df_root(path, aso_sv_kd_unified_bucket,  date, device_code):
    return path.format(aso_sv_kd_unified_bucket, date,  device_code)


aso_sv_raw_parquet_path = get_aso_sv_raw_df_root(aso_sv_raw_bucket_path, aso_sv_kd_raw_bucket,
                                                           date,
                                                           platform)
aso_kd_raw_parquet_path = get_aso_kd_raw_df_root(aso_kd_raw_bucket_path, aso_sv_kd_raw_bucket,
                                                           date,
                                                           platform)

aso_sv_kd_unified_parquet_path = get_aso_sv_kd_unified_df_root(aso_sv_kd_unified_bucket_path,
                                                                   aso_sv_kd_unified_bucket,
                                                                   date,
                                                                   device_code
                                                                   )
def test_android_sv_kd_keyword_count():
    print 'test android all sv keyword count with different country'
    country_iterate="US"
    print country_iterate
    expected_sv_count = spark.read.parquet(aso_sv_raw_parquet_path.split("platform")[0]).filter(
        "platform=2 and device_id=1 and country_code='{}'".format(
            country_iterate)) #.count()
    expected_kd_count = spark.read.parquet(aso_kd_raw_parquet_path.split("platform")[0]).filter(
        "platform=2 and device_id=1 and country_code='{}'".format(
            country_iterate)) #.count()
    
    result=expected_sv_count.join(expected_kd_count, expected_sv_count.keyword_id==expected_kd_count.keyword_id, 'outer').select(expected_kd_count.keyword_id).collect()
    print len(result)
    unified_count = spark.read.parquet(aso_sv_kd_unified_parquet_path.split("device_code")[0]).filter(
        "country_code='{}'and device_code='ios-phone'".format(
            country_iterate)).select(
        "keyword_id").distinct().count()
        
    print unified_count
    
    
def test_android_sv_kd_keyword_count_citus():
    country_code_android_list=['US']
    print 'test android all sv-kd keyword count with different country'
    for country_iterate in country_code_android_list:
        print country_iterate
        expected_sv_count = spark.read.parquet(aso_sv_raw_parquet_path.split("platform")[0]).filter(
            "platform=1 and device_id=0 and country_code='{}'".format(
                country_iterate))
        expected_kd_count = spark.read.parquet(aso_kd_raw_parquet_path.split("platform")[0]).filter(
            "platform=1 and device_id=0 and country_code='{}'".format(
                country_iterate))
        print expected_kd_count

        result = expected_sv_count.join(expected_kd_count,
                                        expected_sv_count.keyword_id == expected_kd_count.keyword_id,
                                        'outer').select(expected_kd_count.keyword_id).collect()
        print len(result)
        sql = "select count(distinct keyword_id) from aso.aso_sv_kd_fact_v1 where device_code='{}' and country_code='{}' and date='{}'".format('android-all',country_iterate,date)
        db_result = get_aso_citus_connection(urn, sql)

        # unified_count = .spark.read.parquet(.aso_sv_kd_unified_parquet_path.split("device_code")[0]).filter(
        #     "country_code='{}' and device_code='android-all'".format(
        #         country_iterate)).select(
        #     "keyword_id").distinct().count()
        print db_result
    
    # selassertNotEqual(expected_sv_count or expected_kd_count, 0)
    # self.assertEqual(max(expected_sv_count, expected_kd_count), unified_count,
    #                  msg='{}~{}'.format(max(expected_sv_count, expected_kd_count), unified_count))

print "########################"

keyword_id=1387
def test_android_sv_kd_value():
    print 'test android all sv-kd keyword count with different country'
    country_code = 'US'
    expected_sv_count = spark.read.parquet(aso_sv_raw_parquet_path.split("platform")[0]).filter(
        "platform=1 and device_id=0 and country_code='{}' and keyword_id={}".format(
            country_code,keyword_id))
    expected_kd_count = spark.read.parquet(aso_kd_raw_parquet_path.split("platform")[0]).filter(
        "platform=1 and device_id=0 and country_code='{}' and keyword_id={}".format(
        country_code, keyword_id))

    result = expected_sv_count.join(expected_kd_count,
                                    expected_sv_count.keyword_id == expected_kd_count.keyword_id,
                                    'outer').collect()[:]

    unified_result = spark.read.parquet(aso_sv_kd_unified_parquet_path.split("device_code")[0]).filter(
        "country_code='{}' and device_code='android-all' and keyword_id={}".format(
            country_code, keyword_id)).collect()[:]
            
    print result
    print unified_result
    print result[0]["search_volume"]
    print unified_result[0]["search_volume"]
    print result[0]["difficulty"]
    print unified_result[0]["difficulty"]
    
keyword_id=1387
def test_android_sv_kd_value_citus():
    print 'test android all sv-kd keyword count with different country'
    sql = "select search_volume, difficulty from aso.aso_sv_kd_fact_v1 where device_code='{}' and country_code='{}' and date='{}' and keyword_id='{}'".format('android-all','US', date,keyword_id)

    country_code = 'US'
    expected_sv_count = spark.read.parquet(aso_sv_raw_parquet_path.split("platform")[0]).filter(
        "platform=1 and device_id=0 and country_code='{}' and keyword_id={}".format(
            country_code,keyword_id))
    expected_kd_count = spark.read.parquet(aso_kd_raw_parquet_path.split("platform")[0]).filter(
        "platform=1 and device_id=0 and country_code='{}' and keyword_id={}".format(
        country_code, keyword_id))

    result = expected_sv_count.join(expected_kd_count,
                                    expected_sv_count.keyword_id == expected_kd_count.keyword_id,
                                    'outer').collect()[:]
    print result
    print result[0]["search_volume"]
    print type(result[0]["search_volume"])
    print result[0]["difficulty"]

    db_result = get_aso_citus_connection(urn, sql)
    print db_result[0]

    
    print abs(decimal.Decimal(round(result[0]["search_volume"],6))-db_result[0][0])<0.01
    print abs(decimal.Decimal(round(result[0]["difficulty"],1))-db_result[0][1])<0.01




mkt_new_bucket = "b2c-prod-data-pipeline-unified-aso"
conf = Conf(
    bucket_name=mkt_new_bucket
)

start_date = '2016-10-11'
end_date = '2019-06-30'

def test_android_ts_date_period_completeness():
    date_list = get_date_list_from_granularity(start_date, end_date, 'daily')
    print date_list
    path_fail, data_fail = dict(), dict()
    path_fail[device_code], data_fail[device_code] = list(), list()
    for date in date_list:
        unified_parquet_path = "unified/app-tech.aso.keyword-difficulty-search-volume.v1/fact/granularity=daily/date={}".format(date) #get_aso_ts_unified_data_path(date, device_code)
        files = S3Bucket(conf=conf).all(unified_parquet_path)
        if files:
            if check_parquet_exist(files):
                data_fail[device_code].append(date)
        else:
            path_fail[device_code].append(date)
    if not path_fail[device_code]:
        path_fail.pop(device_code)
    if not data_fail[device_code]:
        data_fail.pop(device_code)
        
    print data_fail
    print path_fail


def test_android_ts_date_period_completeness_citus():
    date_list = get_date_list_from_granularity(start_date, end_date, 'daily')
    #print date_list
    path_fail, data_fail = dict(), dict()
    path_fail[device_code], data_fail[device_code] = list(), list()
    
    sql = "select distinct date from aso.aso_sv_kd_fact_v1 where device_code='android-all'"
    db_result = get_aso_citus_connection(urn, sql)

    acutal_db_result=[]
    for item in db_result:
        acutal_db_result.append(str(item[0]))
        
        
    print list(set(date_list).difference(set(acutal_db_result)))


def test_kd_do_not_have_zero():
    print 'test no Zero value in keyword diffculity'
    sql = "select * from aso_sv_kd_fact_v1 where device_code = 'android-all' and difficulty = 0"
    db_result = get_aso_citus_connection(urn, sql)
    print db_result

# test_android_sv_kd_keyword_count()
# test_android_sv_kd_keyword_value()
# test_android_ts_date_period_completeness()
test_kd_do_not_have_zero()


In [0]:

import unittest
from random import choice, randint

from aaintdatapipeline.application.app_qa.common.utils import get_month_start_end_date
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import IOS_PHONE, IOS_TABLET, \
    ANDROID_ALL, ASO_TS_RAW_DATA_HEAD
from aaintdatapipeline.application.app_qa.data_validation_v1.pysparktest import PySparkTest
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_random_date, get_aso_ts_raw_df, get_aso_svi_raw_df, get_aso_kd_raw_df,
    check_parquet_exist, get_date_list_from_granularity, get_aso_ts_unified_data_path)
from aaintdatapipeline.core.fs.device import S3Bucket
from aaintdatapipeline.core.conf import Conf
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_aso_citus_connection, datetime_to_string)

from aaintdatapipeline.application.app_qa.data_validation_v1.constants import \
    urn


mkt_new_bucket = "b2c-prod-int-data-pipeline-unified-paid-aso"
ts_unified_path = \
    "s3://{}/unified/app-tech.market.aso-keyword-trafficshare.v1/fact/granularity=daily/date={}/device_code={}/"
svi_kd_unified_path = "s3://{}/unified/app-tech.market.aso-keyword-difficulty-search-volume.v1/fact/" \
                      "granularity=monthly/date={}/device_code={}/"


date = "2018-06-01"
platform = "1"
device_code="ios-tablet"

aso_sv_kd_raw_bucket = "prod.appannie.mktint.data"
aso_sv_raw_bucket_path = "s3://{}/oss/SEARCH_VOLUME/routine/granularity=daily/date={}/platform={}/"
aso_kd_raw_bucket_path = "s3://{}/oss/KEYWORD_DIFFICULTY/routine/granularity=daily/date={}/platform={}"


aso_sv_kd_unified_bucket = "b2c-prod-data-pipeline-unified-aso"
aso_sv_kd_unified_bucket_path = "s3://{}/unified/app-tech.aso.keyword-difficulty-search-volume.v1/fact/granularity=daily/date={}/device_code={}/"




def get_aso_sv_raw_df_root(path, aso_sv_raw_bucket, date, platform):
    return path.format(aso_sv_raw_bucket, date, platform)

def get_aso_kd_raw_df_root(path, aso_kd_raw_bucket, date, platform):
    return path.format(aso_kd_raw_bucket, date, platform)

def get_aso_sv_kd_unified_df_root(path, aso_sv_kd_unified_bucket,  date, device_code):
    return path.format(aso_sv_kd_unified_bucket, date,  device_code)


aso_sv_raw_parquet_path = get_aso_sv_raw_df_root(aso_sv_raw_bucket_path, aso_sv_kd_raw_bucket,
                                                           date,
                                                           platform)
aso_kd_raw_parquet_path = get_aso_kd_raw_df_root(aso_kd_raw_bucket_path, aso_sv_kd_raw_bucket,
                                                           date,
                                                           platform)

aso_sv_kd_unified_parquet_path = get_aso_sv_kd_unified_df_root(aso_sv_kd_unified_bucket_path,
                                                                   aso_sv_kd_unified_bucket,
                                                                   date,
                                                                   device_code
                                                                   )
def test_android_sv_kd_keyword_count():
    print 'test android all sv keyword count with different country'
    country_iterate="US"
    print country_iterate
    expected_sv_count = spark.read.parquet(aso_sv_raw_parquet_path.split("platform")[0]).filter(
        "platform=2 and device_id=1 and country_code='{}'".format(
            country_iterate)) #.count()
    expected_kd_count = spark.read.parquet(aso_kd_raw_parquet_path.split("platform")[0]).filter(
        "platform=2 and device_id=1 and country_code='{}'".format(
            country_iterate)) #.count()
    
    result=expected_sv_count.join(expected_kd_count, expected_sv_count.keyword_id==expected_kd_count.keyword_id, 'outer').select(expected_kd_count.keyword_id).collect()
    print len(result)
    unified_count = spark.read.parquet(aso_sv_kd_unified_parquet_path.split("device_code")[0]).filter(
        "country_code='{}'and device_code='ios-phone'".format(
            country_iterate)).select(
        "keyword_id").distinct().count()
        
    print unified_count
    
    
def test_android_sv_kd_keyword_count_citus():
    country_code_android_list=['US']
    print 'test android all sv-kd keyword count with different country'
    for country_iterate in country_code_android_list:
        print country_iterate
        expected_sv_count = spark.read.parquet(aso_sv_raw_parquet_path.split("platform")[0]).filter(
            "platform=1 and device_id=0 and country_code='{}'".format(
                country_iterate))
        expected_kd_count = spark.read.parquet(aso_kd_raw_parquet_path.split("platform")[0]).filter(
            "platform=1 and device_id=0 and country_code='{}'".format(
                country_iterate))
        print expected_kd_count

        result = expected_sv_count.join(expected_kd_count,
                                        expected_sv_count.keyword_id == expected_kd_count.keyword_id,
                                        'outer').select(expected_kd_count.keyword_id).collect()
        print len(result)
        sql = "select count(distinct keyword_id) from aso.aso_sv_kd_fact_v1 where device_code='{}' and country_code='{}' and date='{}'".format('android-all',country_iterate,date)
        db_result = get_aso_citus_connection(urn, sql)

        # unified_count = .spark.read.parquet(.aso_sv_kd_unified_parquet_path.split("device_code")[0]).filter(
        #     "country_code='{}' and device_code='android-all'".format(
        #         country_iterate)).select(
        #     "keyword_id").distinct().count()
        print db_result
    
    # selassertNotEqual(expected_sv_count or expected_kd_count, 0)
    # self.assertEqual(max(expected_sv_count, expected_kd_count), unified_count,
    #                  msg='{}~{}'.format(max(expected_sv_count, expected_kd_count), unified_count))

print "########################"

keyword_id=1387
def test_android_sv_kd_keyword_value():
    print 'test android all sv-kd keyword count with different country'
    country_code = 'US'
    expected_sv_count = spark.read.parquet(aso_sv_raw_parquet_path.split("platform")[0]).filter(
        "platform=1 and device_id=0 and country_code='{}' and keyword_id={}".format(
            country_code,keyword_id))
    expected_kd_count = spark.read.parquet(aso_kd_raw_parquet_path.split("platform")[0]).filter(
        "platform=1 and device_id=0 and country_code='{}' and keyword_id={}".format(
        country_code, keyword_id))

    result = expected_sv_count.join(expected_kd_count,
                                    expected_sv_count.keyword_id == expected_kd_count.keyword_id,
                                    'outer').collect()[:]

    unified_result = spark.read.parquet(aso_sv_kd_unified_parquet_path.split("device_code")[0]).filter(
        "country_code='{}' and device_code='android-all' and keyword_id={}".format(
            country_code, keyword_id)).collect()[:]
            
    print result
    print unified_result
    print result[0]["search_volume"]
    print unified_result[0]["search_volume"]
    print result[0]["difficulty"]
    print unified_result[0]["difficulty"]

mkt_new_bucket = "b2c-prod-data-pipeline-unified-aso"
conf = Conf(
    bucket_name=mkt_new_bucket
)

start_date = '2016-10-11'
end_date = '2019-06-30'

def test_android_ts_date_period_completeness():
    date_list = get_date_list_from_granularity(start_date, end_date, 'daily')
    print date_list
    path_fail, data_fail = dict(), dict()
    path_fail[device_code], data_fail[device_code] = list(), list()
    for date in date_list:
        unified_parquet_path = "unified/app-tech.aso.keyword-difficulty-search-volume.v1/fact/granularity=daily/date={}".format(date) #get_aso_ts_unified_data_path(date, device_code)
        files = S3Bucket(conf=conf).all(unified_parquet_path)
        if files:
            if check_parquet_exist(files):
                data_fail[device_code].append(date)
        else:
            path_fail[device_code].append(date)
    if not path_fail[device_code]:
        path_fail.pop(device_code)
    if not data_fail[device_code]:
        data_fail.pop(device_code)
        
    print data_fail
    print path_fail
        
# test_android_sv_kd_keyword_count()
# test_android_sv_kd_keyword_value()
# test_android_ts_date_period_completeness()
test_android_sv_kd_keyword_count_citus()

In [0]:
%%sh
# aws s3 ls s3://prod_appannie_appletv/country-ranks/

# aws s3 ls s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2019-10-23/platform=1/
# aws s3 ls s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2019-07-01/platform=1/

# # aws s3 ls s3://prod.appannie.mktint.data/oss/KEYWORD_DIFFICULTY/routine/granularity=daily/date=2019-07-01/platform=1/
 

# aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.keyword-difficulty-search-volume.v1/fact/granularity=daily/date=2019-10-23/device_code=android-all/
aws s3 ls s3://b2c-prod-data-pipeline-raw-aso/unified/app-tech.aso.keyword-difficulty-search-volume.v1/fact/granularity=daily/date=2019-10-23/device_code=android-all/


# # %%sh
# PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF 
# set search_path=aso;
# select * from aso_sv_kd_fact_v1 where device_code='android-all' and date='2019-10-23'

# EOF 



In [0]:

from pyspark.sql.functions import round
# raw data:app_type=SINGLE_APP/
# aws s3 ls s3://prod.appannie.mktint.data/oss/SEARCH_ADS_KEYWORD_SOV/routine/granularity=daily/date=2019-07-26/

# aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.sov-search-ads-keyword-fact.v1/metric/granularity=daily/date=2019-07-25/


# aws s3 ls s3://prod.appannie.mktint.data/oss/SEARCH_ADS_KEYWORD_SOV/routine/granularity=daily/date=2019-08-08/platform=1/app_type=SINGLE_APP/
 
# aws s3 ls s3://prod.appannie.aso.paid.product/oss/DAY/2018-09-27/TS/2/

spark.read.csv("s3://prod.appannie.aso.paid.product/oss/DAY/2018-10-27/TS/2/",sep="\t").filter("_c2='UA' and  _c1=1 and _c0=2").select("_c3" , round('_c6',6).alias('r')).filter("r!=0").select("_c3").distinct().show()


spark.read.parquet("s3://b2c-prod-int-data-pipeline-unified-paid-aso/unified/app-tech.market.aso-keyword-trafficshare.v1/fact/granularity=daily/date=2018-10-27/device_code=ios-phone/").filter("country_code='UA'").show(20,False)

# aws s3 ls s3://b2b-prod-int-data-pipeline-metastore/meta/staging-events/app-tech.aso.sov-search-ads-keyword-fact.v1/2019/08/21/08/





In [0]:

import unittest
from random import choice, randint
from aaintdatapipeline.application.app_qa.common.utils import get_date_list

from aaintdatapipeline.application.app_qa.common.utils import get_month_start_end_date
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import IOS_PHONE, IOS_TABLET, \
    ANDROID_ALL, ASO_TS_RAW_DATA_HEAD
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import \
    urn
from aaintdatapipeline.application.app_qa.data_validation_v1.pysparktest import PySparkTest
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_random_date, get_aso_ts_raw_df, get_aso_svi_raw_df, get_aso_kd_raw_df,
    check_parquet_exist, get_date_list_from_granularity, get_aso_ts_unified_data_path, get_aso_est_unified_data_path,
    get_aso_citus_connection,datetime_to_string)
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import \
APP_STORE_RANK_TABLE_NAME, urn, COUNTRY_CODE_CHECK_LIST, ios_devices

from aaintdatapipeline.core.conf import Conf
from aaintdatapipeline.core.fs.device import S3Bucket
mkt_new_bucket = "b2c-prod-int-data-pipeline-unified-paid-aso"
ts_unified_path = \
    "s3://{}/unified/app-tech.market.aso-keyword-trafficshare.v1/fact/granularity=daily/date={}/device_code={}/"
svi_kd_unified_path = "s3://{}/unified/app-tech.market.aso-keyword-difficulty-search-volume.v1/fact/" \
                      "granularity=monthly/date={}/device_code={}/"
conf = Conf(
    bucket_name=mkt_new_bucket
)

traffic_share_table="aso.aso_keyword_trafficshare_fact_v1"
start_date = '2017-01-01'
end_date = '2018-12-31'
date = get_random_date(start_date, end_date)
random_monthly_date = get_random_date(start_date, end_date)
_, monthly_date = get_month_start_end_date(random_monthly_date)
country_code = 'IN'
raw_ts_data_path = get_aso_ts_raw_df(ASO_TS_RAW_DATA_HEAD, date, 1)

def test_android_ts_metric_est():
    expected_prodID_ts = spark.read.csv(raw_ts_data_path, sep="\t").filter(
        "_c1 = 0 and _c2 = '{}' and _c3 = {}".format(
            country_code, keyword_id)).select('_c4', '_c6').collect()
    sql = "select app_id_list,traffic_share_list from {} where country_code='{}' and keyword_id={} and date='{}' and device_code='{}'".format(
        traffic_share_table, country_code, keyword_id, date, ANDROID_ALL)
    db_result = get_aso_citus_connection(urn, sql)
    print 'test_android_ts_metric_est'

    db_count = db_result[0][0] or db_result[0][1] if len(db_result) else 0
    #assertNotEqual(len(db_count), 0)
    #assertEqual(len(db_result[0][0]),len(db_result[0][1]))
    print db_count
    print "expected_prodID_ts"
    print expected_prodID_ts
    print "db_result"
    print db_result
    for data in range(0, len(expected_prodID_ts)):
        product_id_index = list(db_result[0][0]).index(long(expected_prodID_ts[data][0]))
        print round(float(expected_prodID_ts[data][1]),6)
        print db_result[0][1][product_id_index]

#test_android_ts_metric_est()

def test_ios_date_period_completeness():
    expected_date_list = get_date_list(start_date, end_date)
    date_fail = dict()
    actual_date_list = []
    country_code_list="'US', 'FR', 'DE', 'GB', 'AU', 'BE', 'FI', 'GR', 'JP', 'IT'"
    for device_code in ios_devices:
        date_fail[device_code] = list()
        sql = "select distinct(date) from {} where granularity = 'daily' and device_code = '{}' " \
              "and country_code in ({}) and date between '{}' and '{}'".format(traffic_share_table, device_code,
                                                                              country_code_list, start_date,
                                                                              end_date)

        date_list = get_aso_citus_connection(urn, sql)
        for item in date_list:
            actual_date_list.append(datetime_to_string(item[0]))
        print 'actual_date_list'
        print actual_date_list
        print 'expected_date_list'
        print expected_date_list
        for date in expected_date_list:
            if str(date) not in actual_date_list:
                date_fail[device_code].append(date)
        if not date_fail[device_code]:
            date_fail.pop(device_code)
    print date_fail
    # self.assertFalse(date_fail, msg='{}'.format(date_fail))

test_ios_date_period_completeness()

In [0]:
%%sh


In [0]:
%%sh


In [0]:
%%sh


In [0]:
%%sh


In [0]:


import datetime
from collections import defaultdict
from elasticsearch import Elasticsearch

from aaintdatapipeline.application.app_qa.common.utils import get_random_day_in_last_month, datetime_to_string, \
    get_random_day_in_last_week, get_week_start_end_date, send_data_source_date_email, get_month_start_end_date, \
    string_to_datetime

from aaintdatapipeline.core.loader import es
from aaintdatapipeline.core.urn import Urn
from aaintdatapipeline.core.utils.spark import canned_spark, stop
from dateutil.relativedelta import relativedelta

# country_code_list=["us","gb","jp","fr","au"]
# device_code_list=["ios-phone","android-all","ios-tablet"]
# today = datetime.date.today()
# date = today #- datetime.timedelta(days=1)

urn = Urn(
    namespace='app-qa.db-check.v1',
    owner='app_qa'
)


def check_aso_sov_data():
    params={"project_name":"All_Products"}
    country_code_list = ["us", "gb", "jp", "fr", "au"]
    device_code_list = ["ios-phone", "android-all", "ios-tablet"]
    today = datetime.date.today() - datetime.timedelta(days=5)
    date = today - datetime.timedelta(days=2)
    failed_ids = defaultdict(list)
    project_name = params['project_name']
    if project_name == 'All_Products':
        # check daily
        for device_code in device_code_list:
            for country_code in country_code_list:
                query_body = {
                    "query": {"bool": {"must": [{"match": {"device_code": device_code}}, {"match": {"date": date}}]}},
                    "sort": [{"date": {"order": "desc", "mode": "max"}}]}
                res = check_aso_sov_db(country_code, query_body)
                print "{} - {} - {}".format(date,country_code,res["hits"]["total"])
                if not res["hits"]["total"]:
                    failed_ids["daily {} {}".format(date, device_code)].append(country_code)
                    
        # check weekly and monthly            
        sov_dict_list = get_granularity_date(today)
        print sov_dict_list
        if sov_dict_list:
            for granularity, granularity_date in sov_dict_list.iteritems():
                print granularity
                print granularity_date
                for device_code in device_code_list:
                    for country_code in country_code_list:
                        query_body = {
                            "query": {"bool": {
                                "must": [{"match": {"device_code": device_code}}, {"match": {"date": granularity_date}}]}},
                            "sort": [{"date": {"order": "desc", "mode": "max"}}]}
                        res = check_aso_sov_db(country_code, query_body, granularity=granularity)
                        print res
                        if not res["hits"]["total"]:
                            failed_ids["{} {} {} - ".format(granularity,granularity_date, device_code)].append(country_code)
        return failed_ids


def check_aso_sov_db(country_code, query_body,granularity="daily"):
    common_config = {
        "database": "aso_sov"
    }
    es_conn = es.connection(urn, common_config)
    res = es_conn.search(index="aso-kpi-sov_search_ads_keyword_fact_v1_{}_{}_*".format(granularity,country_code.lower()),
                         body=query_body)
    return res

def get_granularity_date(today):
    print today
    granularity_dic= {}
    if today.isoweekday() == 5:
        week_date = today - datetime.timedelta(days=6)
        granularity_dic["weekly"]=week_date
    if today.day == 6:
        month_date = datetime.date(today.year, today.month, 1) - relativedelta(days=1)
        granularity_dic["monthly"]=month_date
    print granularity_dic
    return granularity_dic

check_aso_sov_data()


In [0]:
%%sh
aws s3 ls s3://prod.appannie.mktint.data/oss/SEARCH_VOLUME/routine/granularity=daily/date=2017-07-06/platform=1/
aws s3 ls s3://prod.appannie.mktint.data/oss/KEYWORD_DIFFICULTY/routine/granularity=daily/date=2017-07-06/platform=1/
aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.keyword-difficulty-search-volume.v1/fact/granularity=daily/date=2017-07-06/device_code=android-all/ 

